# TensorFlow and TensorBoard with Regularization



## Purpose

The purpose of this lab is threefold.  

1.   to review using `TensorFlow` and `TensorBoard` for modeling and evaluation with neural networks
2.   to review using data science pipelines and cross-validation with neural networks
3.   to review using `TensorFlow` for neural network regularization

We'll be continuting our investigation of the canonical [Titanic Data Set](https://www.kaggle.com/competitions/titanic/overview) that we began [previously](https://github.com/learn-co-curriculum/enterprise-paired-nn-eval).

## The Titanic

### The Titanic and it's data



RMS Titanic was a British passenger liner built by Harland and Wolf and operated by the White Star Line. It sank in the North Atlantic Ocean in the early morning hours of 15 April 1912, after striking an iceberg during her maiden voyage from Southampton, England to New York City, USA.

Of the estimated 2,224 passengers and crew aboard, more than 1,500 died, making the sinking one of modern history's deadliest peacetime commercial marine disasters. 

Though there were about 2,224 passengers and crew members, we are given data of about 1,300 passengers. Out of these 1,300 passengers details, about 900 data is used for training purpose and remaining 400 is used for test purpose. The test data has had the survived column removed and we'll use neural networks to predict whether the passengers in the test data survived or not. Both training and test data are not perfectly clean as we'll see.

Below is a picture of the Titanic Museum in Belfast, Northern Ireland.

In [2]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://upload.wikimedia.org/wikipedia/commons/c/c0/Titanic_Belfast_HDR.jpg", width=400, height=400)

### Data Dictionary

*   *Survival* : 0 = No, 1 = Yes
*   *Pclass* : A proxy for socio-economic status (SES)
  *   1st = Upper
  *   2nd = Middle
  *   3rd = Lower
*   *sibsp* : The number of siblings / spouses aboard the Titanic
  *   Sibling = brother, sister, stepbrother, stepsister
  *   Spouse = husband, wife (mistresses and fiancés were ignored)
*   *parch* : The # of parents / children aboard the Titanic
  *   Parent = mother, father
  *   Child = daughter, son, stepdaughter, stepson
  *   Some children travelled only with a nanny, therefore *parch*=0 for them.
*   *Ticket* : Ticket number
*   *Fare* : Passenger fare (British pounds)
*   *Cabin* : Cabin number embarked
*   *Embarked* : Port of Embarkation
  *   C = Cherbourg (now Cherbourg-en-Cotentin), France
  *   Q = Queenstown (now Cobh), Ireland
  *   S = Southampton, England
*   *Name*, *Sex*, *Age* (years) are all self-explanatory

## Libraries and the Data



### Importing libraries

In [3]:
# Load the germane libraries

import pandas as pd
import numpy as np
import seaborn as sns 
from pandas._libs.tslibs import timestamps
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler

import tensorflow as tf
import keras 
from keras import models
from sklearn.impute import SimpleImputer
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.losses import binary_crossentropy
from sklearn.model_selection import GridSearchCV
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.wrappers.scikit_learn import KerasClassifier

# Load the TensorBoard notebook extension and related libraries
%load_ext tensorboard
import datetime

### Loading the data

In [4]:
# Load the data

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# We need to do this for when we mamke our predictions from the test data at the end
ids = test[['PassengerId']]

## EDA and Preprocessing

### Exploratory Data Analysis

You have already performed EDA on this data set. Look back on what you did before or see [here](https://github.com/learn-co-curriculum/enterprise-paired-nn-eval).

Of course, feel free to re-run what you have done before or try out some other EDA as you find useful.

### Preprocessing

Let's do the same prepricessing as before.

In [5]:
# Performing preprocessing on the train and test data will be more effecient if we combine the two date sets.
combined = pd.concat([train, test], axis=0, sort=False)

#Age column
combined['Age'].fillna(combined['Age'].median(),inplace=True) # Age

# Embarked column
combined['Embarked'].fillna(combined['Embarked'].value_counts().index[0], inplace=True) # Embarked
combined['Fare'].fillna(combined['Fare'].median(),inplace=True)

# Class column
d = {1:'1st',2:'2nd',3:'3rd'} #Pclass
combined['Pclass'] = combined['Pclass'].map(d) #Pclass

# Making Age into adult (1) and child (0)
combined['Child'] = combined['Age'].apply(lambda age: 1 if age>=18 else 0) 

# Break up the string that has the title and names
combined['Title'] = combined['Name'].str.split('.').str.get(0)  # output : 'Futrelle, Mrs'
combined['Title'] = combined['Title'].str.split(',').str.get(1) # output : 'Mrs '
combined['Title'] = combined['Title'].str.strip()               # output : 'Mrs'
combined.groupby('Title').count()

# Replace the French titles with Enlgish
french_titles = ['Don', 'Dona', 'Mme', 'Ms', 'Mra','Mlle']
english_titles = ['Mr', 'Mrs','Mrs','Mrs','Mrs','Miss']
for i in range(len(french_titles)):
    for j in range(len(english_titles)):
        if i == j:
            combined['Title'] = combined['Title'].str.replace(french_titles[i],english_titles[j])

# Seperate the titles into "major" and "others", the latter would be, e.g., Reverend
major_titles = ['Mr','Mrs','Miss','Master']
combined['Title'] = combined['Title'].apply(lambda title: title if title in major_titles else 'Others')

#Dropping the Irrelevant Columns
combined.drop(['PassengerId','Name','Ticket','Cabin'], axis=1, inplace=True)

# Getting Dummy Variables and Dropping the Original Categorical Variables
categorical_vars = combined[['Pclass','Sex','Embarked','Title','Child']] # Get Dummies of Categorical Variables
dummies = pd.get_dummies(categorical_vars,drop_first=True)
combined = combined.drop(['Pclass','Sex','Embarked','Title','Child'],axis=1)
combined = pd.concat([combined, dummies],axis=1)

# Separating the data back into train and test sets
test = combined[combined['Survived'].isnull()].drop(['Survived'],axis=1)
train = combined[combined['Survived'].notnull()]

# Training
X_train = train.drop(['Survived'],axis=1)
y_train = train['Survived']

# Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
test = sc.fit_transform(test)

## Neural Network Model

### Building the model

#### Define the model as a pipeline

Let's use the data science pipeline for our neural network model.

As you are now using regularization to guard against high variance, i.e. overfitting the data, in the definition of the model below include *dropout* and/or *l2* regularization. Also, feel free to experiment with different activation functions.

In [6]:
# It will help to define our model in terms of a pipeline
def build_classifier(optimizer):
    classifier = Sequential()
    
    classifier.add(Dense(units=16, activation='relu', input_dim=X_train.shape[1]))
    classifier.add(Dropout(rate=0.2))
    
    classifier.add(Dense(units=16, activation='relu'))
    classifier.add(Dropout(rate=0.2))
    
    classifier.add(Dense(units=1, activation='sigmoid'))
    
    classifier.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return classifier

#### Use grid search to find help you tune the parameters

You can play with optimizers, epochs, and batch sizes. The ones that we're suggesting are not necessarily the best.

In [7]:
# Grid Search
classifier = KerasClassifier(build_fn = build_classifier)
param_grid = dict(optimizer = ['Adam'],
                  epochs=[10, 20, 50],
                  batch_size=[16, 25, 32])
grid = GridSearchCV(estimator=classifier, param_grid=param_grid, scoring='accuracy')
grid_result = grid.fit(X_train, y_train)
best_parameters = grid.best_params_
best_accuracy = grid.best_score_

Epoch 1/10
45/45 [==============================] - 0s 515us/step - loss: 0.6536 - accuracy: 0.6685
Epoch 2/10
45/45 [==============================] - 0s 573us/step - loss: 0.5952 - accuracy: 0.7402
Epoch 3/10
45/45 [==============================] - 0s 516us/step - loss: 0.5626 - accuracy: 0.7556
Epoch 4/10
45/45 [==============================] - 0s 512us/step - loss: 0.5210 - accuracy: 0.7879
Epoch 5/10
45/45 [==============================] - 0s 588us/step - loss: 0.5172 - accuracy: 0.7725
Epoch 6/10
45/45 [==============================] - 0s 484us/step - loss: 0.4947 - accuracy: 0.7809
Epoch 7/10
45/45 [==============================] - 0s 522us/step - loss: 0.4712 - accuracy: 0.7781
Epoch 8/10
45/45 [==============================] - 0s 491us/step - loss: 0.4735 - accuracy: 0.7963
Epoch 9/10
45/45 [==============================] - 0s 503us/step - loss: 0.4627 - accuracy: 0.8020
Epoch 10/10
45/45 [==============================] - 0s 546us/step - loss: 0.4622 - accuracy: 0.7978

45/45 [==============================] - 0s 636us/step - loss: 0.4800 - accuracy: 0.7868
Epoch 7/20
45/45 [==============================] - 0s 518us/step - loss: 0.4745 - accuracy: 0.7980
Epoch 8/20
45/45 [==============================] - 0s 502us/step - loss: 0.4669 - accuracy: 0.7868
Epoch 9/20
45/45 [==============================] - 0s 478us/step - loss: 0.4341 - accuracy: 0.8191
Epoch 10/20
45/45 [==============================] - 0s 514us/step - loss: 0.4462 - accuracy: 0.8149
Epoch 11/20
45/45 [==============================] - 0s 465us/step - loss: 0.4376 - accuracy: 0.8219
Epoch 12/20
45/45 [==============================] - 0s 499us/step - loss: 0.4427 - accuracy: 0.8121
Epoch 13/20
45/45 [==============================] - 0s 530us/step - loss: 0.4268 - accuracy: 0.8275
Epoch 14/20
45/45 [==============================] - 0s 496us/step - loss: 0.4295 - accuracy: 0.8163
Epoch 15/20
45/45 [==============================] - 0s 479us/step - loss: 0.4365 - accuracy: 0.8107
Epoch

45/45 [==============================] - 0s 540us/step - loss: 0.4685 - accuracy: 0.7837
Epoch 8/50
45/45 [==============================] - 0s 467us/step - loss: 0.4644 - accuracy: 0.7907
Epoch 9/50
45/45 [==============================] - 0s 632us/step - loss: 0.4544 - accuracy: 0.7992
Epoch 10/50
45/45 [==============================] - 0s 541us/step - loss: 0.4461 - accuracy: 0.8020
Epoch 11/50
45/45 [==============================] - 0s 555us/step - loss: 0.4585 - accuracy: 0.7879
Epoch 12/50
45/45 [==============================] - 0s 580us/step - loss: 0.4312 - accuracy: 0.8160
Epoch 13/50
45/45 [==============================] - 0s 553us/step - loss: 0.4300 - accuracy: 0.8048
Epoch 14/50
45/45 [==============================] - 0s 564us/step - loss: 0.4402 - accuracy: 0.7978
Epoch 15/50
45/45 [==============================] - 0s 473us/step - loss: 0.4365 - accuracy: 0.8202
Epoch 16/50
45/45 [==============================] - 0s 452us/step - loss: 0.4356 - accuracy: 0.8230
Epoc

45/45 [==============================] - 0s 541us/step - loss: 0.4177 - accuracy: 0.8149
Epoch 39/50
45/45 [==============================] - 0s 542us/step - loss: 0.3947 - accuracy: 0.8289
Epoch 40/50
45/45 [==============================] - 0s 455us/step - loss: 0.3829 - accuracy: 0.8373
Epoch 41/50
45/45 [==============================] - 0s 410us/step - loss: 0.4097 - accuracy: 0.8163
Epoch 42/50
45/45 [==============================] - 0s 454us/step - loss: 0.3929 - accuracy: 0.8247
Epoch 43/50
45/45 [==============================] - 0s 429us/step - loss: 0.3956 - accuracy: 0.8415
Epoch 44/50
45/45 [==============================] - 0s 405us/step - loss: 0.4100 - accuracy: 0.8275
Epoch 45/50
45/45 [==============================] - 0s 461us/step - loss: 0.3994 - accuracy: 0.8303
Epoch 46/50
45/45 [==============================] - 0s 407us/step - loss: 0.3982 - accuracy: 0.8205
Epoch 47/50
45/45 [==============================] - 0s 420us/step - loss: 0.4029 - accuracy: 0.8247
Ep

45/45 [==============================] - 0s 538us/step - loss: 0.4250 - accuracy: 0.8303
Epoch 20/50
45/45 [==============================] - 0s 460us/step - loss: 0.4128 - accuracy: 0.8429
Epoch 21/50
45/45 [==============================] - 0s 426us/step - loss: 0.4208 - accuracy: 0.8331
Epoch 22/50
45/45 [==============================] - 0s 446us/step - loss: 0.4377 - accuracy: 0.8233
Epoch 23/50
45/45 [==============================] - 0s 442us/step - loss: 0.4111 - accuracy: 0.8303
Epoch 24/50
45/45 [==============================] - 0s 491us/step - loss: 0.4107 - accuracy: 0.8289
Epoch 25/50
45/45 [==============================] - 0s 433us/step - loss: 0.4131 - accuracy: 0.8345
Epoch 26/50
45/45 [==============================] - 0s 458us/step - loss: 0.4118 - accuracy: 0.8247
Epoch 27/50
45/45 [==============================] - 0s 442us/step - loss: 0.4094 - accuracy: 0.8331
Epoch 28/50
45/45 [==============================] - 0s 459us/step - loss: 0.4167 - accuracy: 0.8331
Ep

...

23/23 [==============================] - 0s 435us/step - loss: 0.4423 - accuracy: 0.8149
Epoch 17/20
23/23 [==============================] - 0s 541us/step - loss: 0.4540 - accuracy: 0.7966
Epoch 18/20
23/23 [==============================] - 0s 531us/step - loss: 0.4434 - accuracy: 0.8149
Epoch 19/20
23/23 [==============================] - 0s 530us/step - loss: 0.4407 - accuracy: 0.8079
Epoch 20/20
23/23 [==============================] - 0s 463us/step - loss: 0.4654 - accuracy: 0.7994
Epoch 1/20
23/23 [==============================] - 0s 616us/step - loss: 0.6443 - accuracy: 0.6662
Epoch 2/20
23/23 [==============================] - 0s 624us/step - loss: 0.5881 - accuracy: 0.7167
Epoch 3/20
23/23 [==============================] - 0s 522us/step - loss: 0.5440 - accuracy: 0.7433
Epoch 4/20
23/23 [==============================] - 0s 528us/step - loss: 0.5263 - accuracy: 0.7644
Epoch 5/20
23/23 [==============================] - 0s 521us/step - loss: 0.5171 - accuracy: 0.7658
Epoch 6

23/23 [==============================] - 0s 495us/step - loss: 0.4493 - accuracy: 0.8118
Epoch 18/50
23/23 [==============================] - 0s 526us/step - loss: 0.4473 - accuracy: 0.8062
Epoch 19/50
23/23 [==============================] - 0s 481us/step - loss: 0.4404 - accuracy: 0.7963
Epoch 20/50
23/23 [==============================] - 0s 479us/step - loss: 0.4392 - accuracy: 0.8146
Epoch 21/50
23/23 [==============================] - 0s 414us/step - loss: 0.4289 - accuracy: 0.8062
Epoch 22/50
23/23 [==============================] - 0s 435us/step - loss: 0.4435 - accuracy: 0.7949
Epoch 23/50
23/23 [==============================] - 0s 479us/step - loss: 0.4243 - accuracy: 0.8160
Epoch 24/50
23/23 [==============================] - 0s 438us/step - loss: 0.4387 - accuracy: 0.8118
Epoch 25/50
23/23 [==============================] - 0s 498us/step - loss: 0.4323 - accuracy: 0.8048
Epoch 26/50
23/23 [==============================] - 0s 435us/step - loss: 0.4235 - accuracy: 0.8188
Ep

23/23 [==============================] - 0s 931us/step - loss: 0.3950 - accuracy: 0.8331
Epoch 49/50
23/23 [==============================] - 0s 793us/step - loss: 0.3901 - accuracy: 0.8359
Epoch 50/50
23/23 [==============================] - 0s 843us/step - loss: 0.3898 - accuracy: 0.8289
Epoch 1/50
23/23 [==============================] - 0s 775us/step - loss: 0.7986 - accuracy: 0.6073
Epoch 2/50
23/23 [==============================] - 0s 741us/step - loss: 0.7029 - accuracy: 0.6199
Epoch 3/50
23/23 [==============================] - 0s 632us/step - loss: 0.6764 - accuracy: 0.6255
Epoch 4/50
23/23 [==============================] - 0s 764us/step - loss: 0.6367 - accuracy: 0.6676
Epoch 5/50
23/23 [==============================] - 0s 704us/step - loss: 0.6061 - accuracy: 0.7153
Epoch 6/50
23/23 [==============================] - 0s 701us/step - loss: 0.5794 - accuracy: 0.7447
Epoch 7/50
23/23 [==============================] - 0s 783us/step - loss: 0.5614 - accuracy: 0.7574
Epoch 8/5

23/23 [==============================] - 0s 486us/step - loss: 0.4357 - accuracy: 0.8191
Epoch 29/50
23/23 [==============================] - 0s 522us/step - loss: 0.4106 - accuracy: 0.8345
Epoch 30/50
23/23 [==============================] - 0s 522us/step - loss: 0.4274 - accuracy: 0.8247
Epoch 31/50
23/23 [==============================] - 0s 545us/step - loss: 0.4203 - accuracy: 0.8373
Epoch 32/50
23/23 [==============================] - 0s 530us/step - loss: 0.4308 - accuracy: 0.8387
Epoch 33/50
23/23 [==============================] - 0s 457us/step - loss: 0.4210 - accuracy: 0.8261
Epoch 34/50
23/23 [==============================] - 0s 435us/step - loss: 0.4239 - accuracy: 0.8289
Epoch 35/50
23/23 [==============================] - 0s 500us/step - loss: 0.4255 - accuracy: 0.8429
Epoch 36/50
23/23 [==============================] - 0s 572us/step - loss: 0.4233 - accuracy: 0.8345
Epoch 37/50
23/23 [==============================] - 0s 635us/step - loss: 0.4188 - accuracy: 0.8359
Ep

28/28 [==============================] - 0s 459us/step - loss: 0.4827 - accuracy: 0.7811
Epoch 10/50
28/28 [==============================] - 0s 497us/step - loss: 0.4799 - accuracy: 0.7767
Epoch 11/50
28/28 [==============================] - 0s 490us/step - loss: 0.4772 - accuracy: 0.7901
Epoch 12/50
28/28 [==============================] - 0s 515us/step - loss: 0.4645 - accuracy: 0.8070
Epoch 13/50
28/28 [==============================] - 0s 473us/step - loss: 0.4483 - accuracy: 0.8036
Epoch 14/50
28/28 [==============================] - 0s 472us/step - loss: 0.4503 - accuracy: 0.8013
Epoch 15/50
28/28 [==============================] - 0s 514us/step - loss: 0.4634 - accuracy: 0.8070
Epoch 16/50
28/28 [==============================] - 0s 491us/step - loss: 0.4408 - accuracy: 0.8159
Epoch 17/50
28/28 [==============================] - 0s 501us/step - loss: 0.4411 - accuracy: 0.8047
Epoch 18/50
28/28 [==============================] - 0s 540us/step - loss: 0.4333 - accuracy: 0.8182
Ep

#### `TensorBoard`

`TensorBoard` is `TensorFlow`'s visualization toolkit. It is a dashboard that provides visualization and tooling that is needed for machine learning experimentation. The code immediately below will allow us to use TensorBoard.

N.B. When we loaded the libraries, we loaded the TensorBoard notebook extension. (It is the last line of code in the first code chunk.)

In [8]:
# Clear out any prior log data.
!rm -rf logs
# Be careful not to run this command if already have trained your model and you want to use TensorBoard.

# Sets up a timestamped log directory
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(log_dir)


# The callback function, which will be called in the fit()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#### Fitting the optimal model and evaluating with `TensorBoaard`

Define the early stopping callback. Use your best values from grid serarch with `KerasClassifer` and finally fit the model.

In [9]:
# Define the EarlyStopping object
early_stop = EarlyStopping(monitor='val_loss', min_delta=1e-8,
                           verbose=1, patience=5,
                           mode='min')

# Using KerasClassifier
classifier = KerasClassifier(build_fn = build_classifier,
                             optimizer=best_parameters['optimizer'],
                             batch_size=best_parameters['batch_size'],
                             epochs=best_parameters['epochs'])

# Fit the model with the tensorboard_callback
classifier.fit(X_train,
               y_train,
               verbose=1,
               callbacks=[early_stop, tensorboard_callback])


# Warning: If verbose = 0 (silent) or 2 (one line per epoch), then on TensorBoard's Graphs tab there will be an error.
# The other tabs in TensorBoard will still be function, but if you want the graphs then verbose needs to be 1 (progress bar).

Epoch 1/50
 1/28 [>.............................] - ETA: 0s - loss: 0.7633 - accuracy: 0.5312WARNING:tensorflow:From C:\Users\sstallman\AppData\Local\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
28/28 [==============================] - 0s 3ms/step - loss: 0.6820 - accuracy: 0.5892
Epoch 2/50
28/28 [==============================] - 0s 774us/step - loss: 0.6057 - accuracy: 0.6902
Epoch 3/50
28/28 [==============================] - 0s 784us/step - loss: 0.5590 - accuracy: 0.7587
Epoch 4/50
28/28 [==============================] - 0s 717us/step - loss: 0.5255 - accuracy: 0.7834
Epoch 5/50
28/28 [==============================] - 0s 744us/step - loss: 0.5086 - accuracy: 0.7890
Epoch 6/50
28/28 [==============================] - 0s 668us/step - loss: 0.4796 - accuracy: 0.7935
Epoch

28/28 [==============================] - 0s 1ms/step - loss: 0.4297 - accuracy: 0.8249
Epoch 26/50
28/28 [==============================] - 0s 612us/step - loss: 0.4259 - accuracy: 0.8260
Epoch 27/50
28/28 [==============================] - 0s 622us/step - loss: 0.4189 - accuracy: 0.8350
Epoch 28/50
28/28 [==============================] - 0s 806us/step - loss: 0.4135 - accuracy: 0.8384
Epoch 29/50
28/28 [==============================] - 0s 638us/step - loss: 0.4205 - accuracy: 0.8294
Epoch 30/50
28/28 [==============================] - 0s 662us/step - loss: 0.4211 - accuracy: 0.8294
Epoch 31/50
28/28 [==============================] - 0s 654us/step - loss: 0.4184 - accuracy: 0.8227
Epoch 32/50
28/28 [==============================] - 0s 610us/step - loss: 0.4093 - accuracy: 0.8305
Epoch 33/50
28/28 [==============================] - 0s 643us/step - loss: 0.4057 - accuracy: 0.8395
Epoch 34/50
28/28 [==============================] - 0s 701us/step - loss: 0.4149 - accuracy: 0.8260
Epoc

In [10]:
# Call TensorBoard within SaturnCloud [Comment this out if you are not in SaturnCloud]
import os
print(f"https://{os.getenv('SATURN_JUPYTER_BASE_DOMAIN')}/proxy/8000/")
%tensorboard --logdir logs/fit --port 8000 --bind_all
# This will generate a hyperlink. Click on that to open TensorBoard!
# (You'll see a 404 error below the link, just ignore that.)

# Call TensorBoard [Not in SaturnCloud]
# Uncomment the next time if you are not in SC
# %tensorboard --logdir logs/fit

https://None/proxy/8000/


ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 16292.

#### Results and Predictions

Calculate the predictions, save them as a csv, and print them.

In [11]:
# Your code here (use more cells if you need to)
preds = classifier.predict(test)

In [15]:
preds = preds.flatten()

In [17]:
# Create a DataFrame with the PassengerId and the predicted Survived values
submission = pd.DataFrame({
    'PassengerId': ids['PassengerId'],
    'Survived': preds
})

In [18]:
submission.to_csv('submission.csv', index=False)

In [20]:
print(submission.head())
      

   PassengerId  Survived
0          892       0.0
1          893       0.0
2          894       0.0
3          895       0.0
4          896       0.0


Continue to tweak your model until you are happy with the results based on model evaluation.

## Conclusion

Now that you have the `TensorBoard` to help you look at your model, you can better understand how to tweak your model.

How do your predictions compare to what you did last time?

Remember that your "fancier" model may be less accurate... but that is okay if that is the case since we're trying to guard against variance with regularization techniques.